# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

## Лабораторная работа 7

In [1]:
import pandas as pd
import re

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [2]:
recipes = pd.read_csv('ru_recipes_sample.csv', sep = ',')

In [3]:
print(recipes.description[3466])

- И три корочки хлеба! - сделал заказ Буратино в таверне по дороге в Страну Дураков. 

- Ну хотя бы... под соусом болоньез, per favore! - попросил бы Пиноккио. 

Ну, а почему бы и нет? - подумала я. 
 
 "Говорят, под Новый год, 
 Что ни пожелаешь:
 Всe всегда произойдeт, 
 Всe всегда сбывается!...!

 А мы подружим - наших таких одинаковых и таких разных - задорных мальчишек из разных стран и сказок в нашем рецепте. (Лучше поздно, чем никогда, правда ведь?)

 Итак, под Новый год волшебная сказка превращается в настоящее чудо!


In [4]:
recipes.description

0       Этот коктейль готовлю из замороженной клубники...
1                                        Быстро и вкусно.
2             Сытный, овощной салатик, пальчики оближете.
3       Картофельное пюре и куриные котлеты - вкусная ...
4       Вишневая наливка имеет яркий вишневый вкус, ко...
                              ...                        
3462    Для тех, кто любит чечевицу... Вам сюда! Очень...
3463    Баклажановые фантазии продолжаются! Предлагаю ...
3464    Мое любимое блюдо лазанья. Но кушать только фа...
3465    Прошлым летом варила варенье из одуванчиков по...
3466    - И три корочки хлеба! - сделал заказ Буратино...
Name: description, Length: 3467, dtype: object

In [3]:
def delete_description_without_Russians_recipes(text):
    res = re.sub(r'[^\d а-яА-Я]+', ' ', text)
    res = re.sub(r' +', ' ', res)
    res = res.strip()
    return res.lower()

In [5]:
%%time
recipes['description'].map(delete_description_without_Russians_recipes)[3466]

CPU times: user 89.8 ms, sys: 3.53 ms, total: 93.4 ms
Wall time: 90.9 ms


'и три корочки хлеба сделал заказ буратино в таверне по дороге в страну дураков ну хотя бы под соусом болоньез попросил бы пиноккио ну а почему бы и нет подумала я говорят под новый год что ни пожелаешь вс всегда произойд т вс всегда сбывается а мы подружим наших таких одинаковых и таких разных задорных мальчишек из разных стран и сказок в нашем рецепте лучше поздно чем никогда правда ведь итак под новый год волшебная сказка превращается в настоящее чудо'

In [137]:
recipes['description'] = recipes['description'].map(delete_description_without_Russians_recipes)

### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [53]:
from nltk import word_tokenize, edit_distance

In [251]:
text_nltk = ' '.join(recipes.description)
words = word_tokenize(text_nltk)

In [73]:
import numpy as np

In [96]:
random_words = np.random.choice(words, size = (2, 5), replace = False)

In [97]:
random_words

array([['Подходит', 'у', 'и', 'по', 'салате'],
       ['со', 'упустила', 'подается', 'жена', 'сравниться']], dtype='<U38')

In [128]:
for i in range(len(random_words.T)):
    print(f'''d({random_words[0][i]}, {random_words[1][i]}) = \
{edit_distance(random_words[0][i], random_words[1][i])}''')

d(Подходит, со) = 7
d(у, упустила) = 7
d(и, подается) = 8
d(по, жена) = 4
d(салате, сравниться) = 7


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [148]:
!pip install nums_from_string

In [149]:
import nums_from_string

In [249]:
def is_plagiarism(s1: str, s2: str) -> bool:
    
    s1 = word_tokenize(s1)
    s2 = word_tokenize(s2)
    P = 0
    
    for w1 in s1:
        for w2 in s2:
            if edit_distance(w1, w2) < 2:
                P += 1
                break
            
    L = max(len(s1), len(s2))
    print(P, L)
    
    if P/L > 0.8:
        return True
    return False

In [175]:
def nums_from_string_for_url(url):
    return nums_from_string.get_nums(url)[0]

In [188]:
recipes_135488 = recipes.description[recipes['url'].map(nums_from_string_for_url) == 135488].tolist()[0]
recipes_851934 = recipes.description[recipes['url'].map(nums_from_string_for_url) == 851934].tolist()[0]

In [207]:
recipes_851934

'замечательной закуской к напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком куда мы добавим орешков из бородинского хлеба положив в середину маслины закуска получается крайне необычной оригинальной и ценится особенно мужчинами'

In [208]:
recipes_135488

'прекрасной закуской к крепким напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком который мы спрячем в орешках из бородинского хлеба положив в середину маслину закуска получается необычной оригинальной и ценится особенно мужчинами'

In [250]:
is_plagiarism(recipes_135488, recipes_851934)

31 36


True

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [253]:
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()

In [256]:
stem_words = [stemmer.stem(w) for w in words]
norm_words = [morph.parse(w)[0].normalized.word for w in words]

In [285]:
df_word_stem_norm = pd.DataFrame(data = {'stemmed_word': stem_words, 'normalized_word': norm_words},
            index = words)
df_word_stem_norm.rename_axis('word')

,stemmed_word,normalized_word
word,,
этот,этот,этот
коктейль,коктейл,коктейль
готовлю,готовл,готовить
из,из,из
замороженной,заморожен,заморозить
...,...,...
сказка,сказк,сказка
превращается,превраща,превращаться
в,в,в


In [296]:
df_word_stem_norm.iloc[45].tolist()

['картофельн', 'картофельный']

In [ ]:
#понятно, что Стемминг - это нахождение основы слова, то есть без окончания
#на самом деле забавно, что в возвратных глаголах есть СЯ - и это основа, однако мы ее не берем)
#а вот Лемматизация - это когда мы приводим в "нормальную" форму, то есть для
#существительных в Именительный падеж, для глаголов в инфинитив, прилагательные -
# в Именительный падеж, мужской род и тд

5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [298]:
from nltk.corpus import stopwords
stop_rus = stopwords.words('russian')

In [305]:
def description_no_stopwords_in_recipes(text):
    return ' '.join([word for word in word_tokenize(text) if not word in stop_rus])

In [310]:
recipes['description_no_stopwords'] = recipes['description'].map(description_no_stopwords_in_recipes)
recipes.head()

,url,name,ingredients,description,description_no_stopwords
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...",этот коктейль готовлю из замороженной клубники...,коктейль готовлю замороженной клубники клубник...
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...",быстро и вкусно,быстро вкусно
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...",сытный овощной салатик пальчики оближете,сытный овощной салатик пальчики оближете
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...",картофельное пюре и куриные котлеты вкусная кл...,картофельное пюре куриные котлеты вкусная клас...
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...",вишневая наливка имеет яркий вишневый вкус кот...,вишневая наливка имеет яркий вишневый вкус кот...


In [323]:
1 - len(word_tokenize(' '.join(recipes['description_no_stopwords'])))/len(
    word_tokenize(' '.join(recipes['description'])))

0.3247968763457839

In [313]:
from collections import Counter

In [324]:
Counter(word_tokenize(' '.join(recipes['description']))).most_common(10)

[('и', 5062),
 ('в', 2592),
 ('с', 1947),
 ('на', 1658),
 ('очень', 1626),
 ('не', 1523),
 ('из', 1023),
 ('я', 982),
 ('а', 876),
 ('рецепт', 874)]

In [325]:
Counter(word_tokenize(' '.join(recipes['description_no_stopwords']))).most_common(10)

[('очень', 1626),
 ('рецепт', 874),
 ('это', 743),
 ('блюдо', 527),
 ('вкусный', 461),
 ('просто', 439),
 ('вкусно', 380),
 ('приготовить', 344),
 ('вкус', 325),
 ('салат', 318)]

### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [340]:
def pancakes(text):
    if 'оладьи' in text.lower():
        return True
    return False

In [351]:
index_pancakes = recipes[recipes['name'].map(pancakes)].index.tolist()
index_pancakes[:5]

[115, 135, 442, 464, 599]

In [353]:
random_pancakes = np.random.choice(index_pancakes, size = 5, replace = False)
random_pancakes

array([3244,  599, 2518, 1139, 2830])

In [360]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [361]:
cv = TfidfVectorizer()
cv.fit(recipes[np.isin(recipes.index, random_pancakes)].description.tolist())

TfidfVectorizer()

In [369]:
cv_df = pd.DataFrame(cv.transform(recipes[np.isin(recipes.index, random_pancakes)].description.tolist()).toarray(),
                    columns = cv.vocabulary_,
                    index = recipes[np.isin(recipes.index, random_pancakes)].name.tolist())
cv_df

,оладьи,из,брокколи,сыра,сытно,как,вкусно,если,подать,со,...,его,несколькими,ингредиентами,именно,картофельно,творожные,получаются,нежными,необычными,очень
Оладьи из брокколи с сыром,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.257378,0.000000,0.000000,0.257378,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Куриные оладьи,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.344041,0.000000,0.000000,0.000000,0.000000
"Оладьи ""Лето""",0.000000,0.139885,0.000000,0.139885,0.112858,0.139885,0.000000,0.139885,0.000000,0.000000,...,0.000000,0.000000,0.139885,0.000000,0.139885,0.000000,0.000000,0.139885,0.139885,0.139885
Оладьи из печени,0.130927,0.000000,0.130927,0.000000,0.105631,0.000000,0.000000,0.000000,0.130927,0.000000,...,0.000000,0.130927,0.000000,0.000000,0.000000,0.000000,0.130927,0.000000,0.000000,0.000000
Картофельно-творожные оладьи,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.208278,0.000000,0.000000,0.208278,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [366]:
from scipy.spatial.distance import cosine

In [368]:
iamtired_df = pd.DataFrame(
    columns = recipes[np.isin(recipes.index, random_pancakes)].name.tolist(), 
    index = recipes[np.isin(recipes.index, random_pancakes)].name.tolist())
iamtired_df

,Оладьи из брокколи с сыром,Куриные оладьи,"Оладьи ""Лето""",Оладьи из печени,Картофельно-творожные оладьи
Оладьи из брокколи с сыром,NaN,NaN,NaN,NaN,NaN
Куриные оладьи,NaN,NaN,NaN,NaN,NaN
"Оладьи ""Лето""",NaN,NaN,NaN,NaN,NaN
Оладьи из печени,NaN,NaN,NaN,NaN,NaN
Картофельно-творожные оладьи,NaN,NaN,NaN,NaN,NaN


In [376]:
for i in cv_df.index.tolist():
    for j in cv_df.index.tolist():
        
        iamtired_df.loc[i, j] = cosine(cv_df.loc[j].tolist(), cv_df.loc[i].tolist())

In [377]:
iamtired_df

,Оладьи из брокколи с сыром,Куриные оладьи,"Оладьи ""Лето""",Оладьи из печени,Картофельно-творожные оладьи
Оладьи из брокколи с сыром,0,1.0,0.944269,0.978066,0.951914
Куриные оладьи,1.0,0,0.978415,0.919188,1.0
"Оладьи ""Лето""",0.944269,0.978415,0,0.907535,0.949004
Оладьи из печени,0.978066,0.919188,0.907535,0,0.94675
Картофельно-творожные оладьи,0.951914,1.0,0.949004,0.94675,0


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [453]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    np.fill_diagonal(sim_df.values, 2022)
    min_pancake = sim_df.values.argmin()

    col = sim_df.columns[min_pancake // sim_df.shape[0]]
    ind = sim_df.index[min_pancake % sim_df.shape[1]]
    
    return col, ind

In [454]:
find_closest(iamtired_df)

('Оладьи "Лето"', 'Оладьи из печени')

In [456]:
recipes[np.isin(recipes.name, find_closest(iamtired_df))].description

2518    драники или деруны уже давно стали интернацион...
2830    31 декабря рабочий день времени на подготовку ...
Name: description, dtype: object

In [459]:
recipes.loc[2518].description

'драники или деруны уже давно стали интернациональным блюдом уходя корнями в белорусскую кухню они широко распространились по всей планете и каждый внес в рецепт свои изменения но основа остается неизменной картофель натертый на терке вот и я не упустила шанс сделать оладьи из картофеля более интересными и вкусными использовали дополнительный ингредиент свежую зелень что придало драникам оладушкам еще больше нежности и вкуса'

In [460]:
recipes.loc[2830].description

'31 декабря рабочий день времени на подготовку праздничного стола ооочень мало на помощь приходят рецепты которые можно приготовить заранее рецепт оладушек простой но их не стыдно подать на праздничный стол тем более несложный процесс приготовления упростился окончательно сейчас на смену мясорубке пришел блендер или кухонный комбайн в чашу которого можно сложить все ингредиенты одновременно вуаля и вкусная закуска готова'

In [ ]:
#ну что ж, на самом деле я бы не сказала что они супер похожи друг на друга НО
#здесь есть упоминание что блюдо вкусное, слово "ингридиенты"